In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
BATCH_SIZE = 16
IMAGE_SIZE = (224, 224)
NUM_CLASSES = 15

train_ds = image_dataset_from_directory(
    'sdc/train',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

val_ds = image_dataset_from_directory(
    'sdc/test',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

train_ds = train_ds.map(lambda x, y: (tf.keras.applications.efficientnet.preprocess_input(x), y))
val_ds = val_ds.map(lambda x, y: (tf.keras.applications.efficientnet.preprocess_input(x), y))

Found 21000 files belonging to 15 classes.
Found 2715 files belonging to 15 classes.


In [3]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)


In [4]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
callbacks = [  
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks
)

Epoch 1/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 635s 479ms/step - accuracy: 0.3465 - loss: 2.0567 - val_accuracy: 0.4729 - val_loss: 1.7285 - learning_rate: 0.0010
Epoch 2/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 570s 434ms/step - accuracy: 0.5187 - loss: 1.5039 - val_accuracy: 0.5002 - val_loss: 1.6057 - learning_rate: 0.0010
Epoch 3/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 486s 370ms/step - accuracy: 0.5936 - loss: 1.2627 - val_accuracy: 0.5227 - val_loss: 1.5971 - learning_rate: 0.0010
Epoch 4/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 463s 353ms/step - accuracy: 0.6547 - loss: 1.0749 - val_accuracy: 0.5470 - val_loss: 1.5535 - learning_rate: 0.0010
Epoch 5/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 392s 299ms/step - accuracy: 0.6944 - loss: 0.9338 - val_accuracy: 0.5617 - val_loss: 1.5532 - learning_rate: 0.0010
Epoch 6/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 430s 327ms/step - accuracy: 0.7329 - loss: 0.8261 - val_accuracy: 0.5661 - val_loss: 1.5515 - learning_rate: 0.0010
Epoch 7/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 437s 333ms

In [6]:
train_loss, train_acc = model.evaluate(train_ds)
val_loss, val_acc = model.evaluate(val_ds)

print(f"Train Accuracy: {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")

1313/1313 ━━━━━━━━━━━━━━━━━━━━ 391s 298ms/step - accuracy: 0.9732 - loss: 0.1207
170/170 ━━━━━━━━━━━━━━━━━━━━ 56s 328ms/step - accuracy: 0.6406 - loss: 1.4875
Train Accuracy: 0.9761
Validation Accuracy: 0.6368


In [7]:
model.save("EfficientNetB0.h5")